# 第5章：将Strands代理部署到生产环境

## 部署简介

在本地开发完Strands代理后，下一步是将它们部署到生产环境。在本章中，我们将专注于AWS Lambda，它为运行AI代理提供了一个出色的无服务器平台。

我们将涵盖：

- AWS Lambda对Strands代理的优势

- 为部署准备您的代理

- 设置API Gateway以实现网络访问

- 监控和扩展注意事项

在本章中，我们将按照课程要求使用Claude 3.7 Sonnet模型（`us.anthropic.claude-3-7-sonnet-20250219-v1:0`）。

## 为什么选择AWS Lambda部署Strands代理？

AWS Lambda为部署Strands代理提供了多项优势：

1. **无服务器架构**：无需服务器管理

2. **按使用量付费**：仅为实际计算时间付费

3. **自动扩展**：轻松处理不同负载

4. **AWS集成**：与其他AWS服务无缝集成

5. **Python支持**：原生支持基于Python的应用程序

这些优势使Lambda成为部署具有可变使用模式的AI代理的理想选择。

## 准备您的Agent用于Lambda

让我们创建一个简单的天气代理，我们将准备部署它：

In [1]:
from strands import Agent, tool

@tool
def weather_info(location: str) -> str:
    """
    Get weather information for a location.
    
    Args:
        location (str): City or location name
        
    Returns:
        str: Weather information for the location
    """
    # Mock data for demonstration
    weather_data = {
        "new york": "72°F, Partly Cloudy",
        "london": "64°F, Rainy",
        "tokyo": "79°F, Sunny"
    }
    
    return weather_data.get(location.lower(), "Weather information not available")

# Create the agent
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[weather_info],
    system_prompt="You are a helpful weather assistant."
)

# Test locally
response = agent("What's the weather like in Tokyo?")

I can help you check the current weather in Tokyo. Let me get that information for you.
Tool #1: weather_info
Currently in Tokyo it's 79°F with sunny conditions. It looks like a nice day there!

现在让我们通过创建Lambda处理函数来构建我们的代码以便部署到Lambda：

In [2]:
%%writefile lambda_function.py
import json
from strands import Agent, tool

@tool
def weather_info(location: str) -> str:
    """
    Get weather information for a location.
    
    Args:
        location (str): City or location name
        
    Returns:
        str: Weather information for the location
    """
    # Mock data for demonstration
    weather_data = {
        "new york": "72°F, Partly Cloudy",
        "london": "64°F, Rainy",
        "tokyo": "79°F, Sunny"
    }
    
    return weather_data.get(location.lower(), "Weather information not available")

# Create the agent outside the handler to benefit from container reuse
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[weather_info],
    system_prompt="You are a helpful weather assistant."
)

def lambda_handler(event, context):
    """
    AWS Lambda handler function
    """
    try:
        # Extract the user message
        body = json.loads(event.get('body', '{}'))
        user_message = body.get('message', '')
        
        if not user_message:
            return {
                'statusCode': 400,
                'body': json.dumps({'error': 'No message provided'})
            }
        
        # Process with our agent
        response = agent(user_message)
        
        return {
            'statusCode': 200,
            'body': json.dumps({'message': response.message})
        }
        
    except Exception as e:
        # Log the error
        print(f"Error: {str(e)}")
        
        return {
            'statusCode': 500,
            'body': json.dumps({'error': 'Internal server error'})
        }

Writing lambda_function.py


## Bedrock 的 IAM 权限

要在 Lambda 中使用 Amazon Bedrock 模型，您需要设置适当的 IAM 权限。以下是一个示例 IAM 策略：

In [3]:
%%writefile trust-policy.json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

Writing trust-policy.json


In [5]:
%%writefile bedrock-policy.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": ["bedrock:InvokeModel", "bedrock:InvokeModelWithResponseStream"],
            "Resource": ["arn:aws:bedrock:*:*:foundation-model/amazon.nova-lite-v1:0", "arn:aws:bedrock:us-east-1:434465421667:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0"]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        }
    ]
}

Overwriting bedrock-policy.json


In [6]:
%%writefile iam_role_policy.json
# Get account id
ACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)

# Create IAM role
aws iam create-role \
  --role-name strands-agents-lambda-bedrock-role \
  --assume-role-policy-document file://trust-policy.json

# Create IAM policy
aws iam create-policy \
  --policy-name strands-agents-lambda-bedrock-policy \
  --policy-document file://bedrock-policy.json

# Attach IAM policy
aws iam attach-role-policy \
  --role-name strands-agents-lambda-bedrock-role \
  --policy-arn arn:aws:iam::${ACCOUNT_ID}:policy/strands-agents-lambda-bedrock-policy

Writing iam_role_policy.json


In [7]:
!sh iam_role_policy.json

{
    "Role": {
        "Path": "/",
        "RoleName": "strands-agents-lambda-bedrock-role",
        "RoleId": "AROAWKKBIHFR5PM6RS67N",
        "Arn": "arn:aws:iam::434465421667:role/strands-agents-lambda-bedrock-role",
        "CreateDate": "2025-06-01T16:26:48+00:00",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "lambda.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }
    }
}
{
    "Policy": {
        "PolicyName": "strands-agents-lambda-bedrock-policy",
        "PolicyId": "ANPAWKKBIHFRWMHLWPIRD",
        "Arn": "arn:aws:iam::434465421667:policy/strands-agents-lambda-bedrock-policy",
        "Path": "/",
        "DefaultVersionId": "v1",
        "AttachmentCount": 0,
        "PermissionsBoundaryUsageCount": 0,
        "IsAt

## Creating a Deployment Package

For Lambda, you need to package your code and dependencies into a ZIP file. Here's a script to create the deployment package:

In [8]:
!sudo yum install zip -y
!sudo apt install zip -y

sudo: yum: command not found
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zip
0 upgraded, 1 newly installed, 0 to remove and 50 not upgraded.
Need to get 176 kB of archives.
After this operation, 544 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2 [176 kB]
Fetched 176 kB in 1s (339 kB/s)m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package zip.
(Reading database ... 16516 files and directories currently installed.)
Preparing to unpack .../zip_3.0-12build2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking zip (3.0-12build2) ...
7Progress: [ 40%] [#######################...................................] 8Setting

In [9]:
%%writefile create_package.sh
#!/bin/bash

# Create a temporary directory
mkdir -p package
cp lambda_function.py package/

# Create a virtual environment and install dependencies
python -m venv venv
. venv/bin/activate
pip install strands-agents -t package/

# Create the ZIP file
cd package
zip -r ../deployment.zip .
cd ..

# Clean up
rm -rf package venv
echo "Deployment package created: deployment.zip"

Writing create_package.sh


In [10]:
!sh create_package.sh

  Using cached strands_agents-0.1.6-py3-none-any.whl.metadata (10 kB)
  Using cached docstring_parser-0.15-py3-none-any.whl.metadata (2.4 kB)
  Using cached mcp-1.9.2-py3-none-any.whl.metadata (28 kB)
  Using cached opentelemetry_api-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.33.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached pydantic-2.11.5-py3-none-any.whl.metadata (67 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached sse_starlette-2.3.6-py3-none-any.whl.metadata (10 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.33.1-py3-none-any.whl.metadata (1.9 kB)
  Using cached openteleme

## 部署到AWS Lambda

以下是如何使用AWS CLI部署您的函数：

In [11]:
%%writefile deploy.sh
#!/bin/bash

# Configuration
ACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)
FUNCTION_NAME="strands_agents_lambda"
ROLE_ARN="arn:aws:iam::${ACCOUNT_ID}:role/strands-agents-lambda-bedrock-role"  # Replace with your role ARN
REGION="us-east-1"

# Create the Lambda function
aws lambda create-function \
    --function-name $FUNCTION_NAME \
    --runtime python3.12 \
    --handler lambda_function.lambda_handler \
    --timeout 30 \
    --memory-size 512 \
    --role $ROLE_ARN \
    --zip-file fileb://deployment.zip \
    --region $REGION

Writing deploy.sh


In [12]:
!sh deploy.sh

{
    "FunctionName": "strands_agents_lambda",
    "FunctionArn": "arn:aws:lambda:us-east-1:434465421667:function:strands_agents_lambda",
    "Runtime": "python3.12",
    "Role": "arn:aws:iam::434465421667:role/strands-agents-lambda-bedrock-role",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 25344795,
    "Description": "",
    "Timeout": 30,
    "MemorySize": 512,
    "LastModified": "2025-06-01T16:27:50.637+0000",
    "CodeSha256": "iFpJ41BCn2UC/n9n750f6c05E6z+lqHJwDMl7A+mybE=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "a3fb99a6-306a-4af5-8624-4b60d4986f3a",
    "State": "Pending",
    "StateReason": "The function is being created.",
    "StateReasonCode": "Creating",
    "PackageType": "Zip",
    "Architectures": [
        "x86_64"
    ],
    "EphemeralStorage": {
        "Size": 512
    },
    "SnapStart": {
        "ApplyOn": "None",
        "OptimizationStatus": "Off"
    },
    "RuntimeVersionConf

## 测试已部署的Agent

部署完成后，您可以通过发送请求来测试您的代理：

In [14]:
import boto3
import json

def invoke_weather_agent(location_query):
    """
    Invokes the strands_agents_lambda with a weather query
    
    Args:
        location_query (str): The weather query (e.g. "What's the weather in Tokyo?")
    
    Returns:
        dict: The response from the Lambda function
    """
    # Create Lambda client
    lambda_client = boto3.client('lambda', region_name='us-east-1')

    try:
        # Invoke the Lambda function
        response = lambda_client.invoke(
            FunctionName='strands_agents_lambda',
            InvocationType='RequestResponse',  # Synchronous call
            Payload=json.dumps({
                'body': json.dumps({
                    'message': location_query
                })
            })
        )
        
        # Parse the response
        payload = json.loads(response['Payload'].read().decode('utf-8'))
        return {
            'status_code': response['StatusCode'],
            'response': json.loads(payload.get('body', '{}'))
        }
        
    except Exception as e:
        print(f"Error invoking Lambda: {str(e)}")
        return {
            'status_code': 500,
            'error': str(e)
        }

# Example usage
if __name__ == "__main__":
    result = invoke_weather_agent("What's the weather like in Tokyo?")
    print("Lambda Response:\n\n", result['response']['message']['content'][0]['text'])

KeyError: 'message'

## Advanced Deployment Options

For production deployments, consider these advanced options:

### 1. API Gateway
Setting up API Gateway to expose your agent via HTTP.

### 2. AWS CDK

The [AWS Cloud Development Kit (CDK)](https://aws.amazon.com/cdk/) allows you to define infrastructure using Python code.

### 3. Serverless Framework

The [Serverless Framework](https://www.serverless.com/) simplifies deploying Lambda functions and related resources:

### 4. Lambda Layers

For large dependencies, you can use [Lambda Layers](https://docs.aws.amazon.com/lambda/latest/dg/configuration-layers.html) to separate dependencies from your function code.

## Monitoring and Scaling

### CloudWatch Metrics and Logs

AWS Lambda automatically sends metrics to CloudWatch, including invocation count, duration, and errors. Set up CloudWatch Alarms to alert you of potential issues.

### Lambda Concurrency

Lambda automatically scales based on the number of incoming requests. For predictable scaling:
- **Reserved Concurrency**: Guarantees a certain number of concurrent executions
- **Provisioned Concurrency**: Keeps functions initialized for immediate response

### Cold Starts

The first invocation of a Lambda function (or after scaling) may experience a "cold start" delay. To mitigate this:
- Use Provisioned Concurrency
- Keep function size small
- Use warm-up pings for critical functions

### Cost Optimization

Lambda charges based on:
- Number of requests
- Duration × memory allocation

Optimize by:
- Tuning memory allocation
- Minimizing function duration
- Using CloudWatch Logs Insights to identify inefficient functions

## Summary

In this chapter, we've covered:

1. Why AWS Lambda is well-suited for deploying Strands Agents
2. How to structure your agent code for Lambda deployment
3. Creating a deployment package with all dependencies
4. Advanced deployment options like Serverless Framework and AWS CDK
5. Monitoring and scaling considerations for production deployments

With these tools and techniques, you can deploy Strands Agents to a production environment that is scalable, cost-effective, and easy to maintain.

In the next chapter, we'll explore advanced patterns for multi-agent systems that can work together to solve complex problems.

## Exercises

1. Modify the example agent to use a different Strands tool and deploy it to Lambda
2. Implement a basic authentication mechanism for your API Gateway endpoint
3. Set up CloudWatch Alarms to monitor your deployed agent
4. Create a Lambda Layer for the Strands dependencies
5. Implement a WebSocket-based solution for streaming agent responses